In [1]:
import numpy as np
import os
import scipy.io
from sklearn.metrics import classification_report,confusion_matrix

In [2]:
#Please add the folder name of the dataset to run it on different dataset.
dataset = 'AWA2'
path = 'E:/Sushree/Dataset/data/xlsa17/data/'

res101 = scipy.io.loadmat(path + dataset + '/res101.mat')
att_splits = scipy.io.loadmat(path + dataset + '/att_splits.mat')

In [3]:
# total number of instances or images = 37322: ranges from 0 to 37321

trainval_loc = np.squeeze(att_splits['trainval_loc']-1) # -1: to consider the overflow problem
print(np.unique(trainval_loc), np.max(np.unique(trainval_loc))) # smallest location: 2, largest location 37321

test_seen_loc = np.squeeze(att_splits['test_seen_loc']-1)
print(np.unique(test_seen_loc), np.max(np.unique(test_seen_loc))) # smallest location: 0, largest location 37319

test_unseen_loc = np.squeeze(att_splits['test_unseen_loc']-1)
print(np.unique(test_unseen_loc), np.max(np.unique(test_unseen_loc))) # smallest location: 1046, largest location 35290



[    2     4     5 ... 37318 37320 37321] 37321
[    0     1     3 ... 37306 37307 37319] 37319
[ 1046  1047  1048 ... 35288 35289 35290] 35290


In [4]:

labels = res101['labels']# direct class labels
print('labels', labels, labels.shape)# 37322 x 1

print('unique_labels', np.unique(labels), np.unique(labels).shape)# class labels range from 1 to 50, 50 classes

# get the labels for trainval, test seen and test unseen sets

labels_trainval = labels[trainval_loc]
print('labels_trainval', labels_trainval, labels_trainval.shape)

unique_labels_trainval = np.unique(labels_trainval) # labels min:1 max:49
print('unique_labels_trainval', unique_labels_trainval, unique_labels_trainval.shape)# 40 classes


labels_test_seen = labels[test_seen_loc]
print('labels_test_seen', labels_test_seen, labels_test_seen.shape)

unique_labels_test_seen = np.unique(labels_test_seen) # labels min:1 max:49
print('unique_labels_test_seen', unique_labels_test_seen, unique_labels_test_seen.shape)# 40 classes


labels_test_unseen = labels[test_unseen_loc]
print('labels_test_unseen', labels_test_unseen, labels_test_unseen.shape)

unique_labels_test_unseen = np.unique(labels_test_unseen) # labels min:7 max:50
print('unique_labels_test_unseen', unique_labels_test_unseen, unique_labels_test_unseen.shape)# 10 classes


if len(labels) == len(labels_trainval) + len(labels_test_seen) + len(labels_test_unseen):
    print('correct number of instances for training, test seen and test unseen categories')
    
print("Number of overlapping classes between trainval and test seen:",len(set(unique_labels_trainval).intersection(set(unique_labels_test_seen))))

print("Number of overlapping classes between trainval and test unseen:",len(set(unique_labels_trainval).intersection(set(unique_labels_test_unseen))))


labels [[ 1]
 [ 1]
 [ 1]
 ...
 [38]
 [38]
 [38]] (37322, 1)
unique_labels [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50] (50,)
labels_trainval [[43]
 [22]
 [43]
 ...
 [40]
 [19]
 [46]] (23527, 1)
unique_labels_trainval [ 1  2  3  4  5  6  8 10 11 12 13 14 15 16 17 18 19 20 21 22 25 26 27 28
 29 32 33 35 36 37 38 39 40 42 43 44 45 46 48 49] (40,)
labels_test_seen [[22]
 [49]
 [14]
 ...
 [25]
 [15]
 [27]] (5882, 1)
unique_labels_test_seen [ 1  2  3  4  5  6  8 10 11 12 13 14 15 16 17 18 19 20 21 22 25 26 27 28
 29 32 33 35 36 37 38 39 40 42 43 44 45 46 48 49] (40,)
labels_test_unseen [[30]
 [30]
 [30]
 ...
 [47]
 [47]
 [47]] (7913, 1)
unique_labels_test_unseen [ 7  9 23 24 30 31 34 41 47 50] (10,)
correct number of instances for training, test seen and test unseen categories
Number of overlapping classes between trainval and test seen: 40
Number of overlapping classes between trainv

In [5]:
X_features = res101['features']

# locations are already subtracted by 1, so they range from 0 to 37321
trainval_vec = X_features[:, trainval_loc].transpose()
test_seen_vec = X_features[:, test_seen_loc].transpose()
test_unseen_vec = X_features[:, test_unseen_loc].transpose()

print("Features for trainval:", trainval_vec.shape) #(23527, 2048)
print("Features for test seen:", test_seen_vec.shape)# (5882, 2048)
print("Features for test unseen:", test_unseen_vec.shape) #(7913, 2048)

Features for trainval: (23527, 2048)
Features for test seen: (5882, 2048)
Features for test unseen: (7913, 2048)


In [6]:
signature = att_splits['att']
print(signature.shape) #(85, 50)

attribute = signature.transpose()
print(attribute, attribute.shape)#(50, 85)

#attribute[attribute<0]=0
#print(attribute, attribute.shape)#(50, 85)

# attribute is defined for all 50 classes, so we cant use locations directly, instead we have to use labels 
# that range from 1 to 50, so we have to subtract 1

train_attributes = np.zeros((len(trainval_loc), 85))
for i in range(len(trainval_loc)):
    train_attributes[i] = attribute[int(labels_trainval[i])-1]

print(train_attributes, train_attributes.shape)# (23527, 85)

test_seen_attributes = np.zeros((len(test_seen_loc), 85))
for i in range(len(test_seen_loc)):
    test_seen_attributes[i] = attribute[int(labels_test_seen[i])-1]

print(test_seen_attributes, test_seen_attributes.shape)# (5882, 85)

test_unseen_attributes = np.zeros((len(test_unseen_loc), 85))
for i in range(len(test_unseen_loc)):
    test_unseen_attributes[i] = attribute[int(labels_test_unseen[i])-1]

print(test_unseen_attributes, test_unseen_attributes.shape)# (7913, 85)


(85, 50)
[[-0.00375358 -0.00375358 -0.00375358 ...  0.00882092  0.03640974
   0.03145501]
 [ 0.12045618  0.00426584  0.         ...  0.17996306  0.0618086
   0.03495531]
 [ 0.26584459  0.20652363  0.         ...  0.05026822  0.04274552
   0.04915256]
 ...
 [ 0.22516498  0.15266022  0.         ...  0.12733492  0.10009694
   0.01771   ]
 [ 0.19613947  0.1966714   0.         ...  0.01787277  0.06698743
   0.25883601]
 [ 0.03819588  0.08046548  0.10363715 ...  0.01479997  0.05250999
   0.14194515]] (50, 85)
[[0.00575881 0.003829   0.         ... 0.03639079 0.13208508 0.01148699]
 [0.         0.00507555 0.         ... 0.15675628 0.09070911 0.01425057]
 [0.00575881 0.003829   0.         ... 0.03639079 0.13208508 0.01148699]
 ...
 [0.         0.06587863 0.         ... 0.02108505 0.10218637 0.0332632 ]
 [0.0084177  0.01262655 0.         ... 0.02104426 0.04138142 0.02316552]
 [0.03877321 0.15834626 0.         ... 0.1760296  0.07107783 0.30279846]] (23527, 85)
[[0.         0.00507555 0.         

In [7]:
# as labels range from 1 to 50, we have subtract 1

trainval_sig = signature[:, (unique_labels_trainval)-1]
test_seen_sig = signature[:, (unique_labels_test_seen)-1]
test_unseen_sig = signature[:, (unique_labels_test_unseen)-1]

print("Signature for trainval:", trainval_sig.shape)
print("Signature for test seen:", test_seen_sig.shape)
print("Signature for test unseen:", test_unseen_sig.shape)

Signature for trainval: (85, 40)
Signature for test seen: (85, 40)
Signature for test unseen: (85, 10)


In [8]:
# by doing this modification, we are changing the range of trainval and test seen labels from 0 to 39 
# and test unseen labels from 0 to 9

k = 0
new_labels_trainval = np.zeros((len(labels_trainval), 1), dtype = 'int')
for labels in unique_labels_trainval:
    new_labels_trainval[labels_trainval == labels] = k
    k = k+1
    
print(new_labels_trainval, new_labels_trainval.shape)#(23527, 1)

l = 0
new_labels_test_seen = np.zeros((len(labels_test_seen), 1), dtype = 'int')
for labels in unique_labels_test_seen:
    new_labels_test_seen[labels_test_seen == labels] = l
    l = l+1
    
print(new_labels_test_seen, new_labels_test_seen.shape)# (5882, 1)

m = 0
new_labels_test_unseen = np.zeros((len(labels_test_unseen), 1), dtype = 'int')
for labels in unique_labels_test_unseen:
    new_labels_test_unseen[labels_test_unseen == labels] = m
    m = m+1  

print(new_labels_test_unseen, new_labels_test_unseen.shape) #  (7913, 1)  


print(np.unique(new_labels_trainval), np.unique(new_labels_trainval).shape)

print(np.unique(new_labels_test_seen), np.unique(new_labels_test_seen).shape)

print(np.unique(new_labels_test_unseen), np.unique(new_labels_test_unseen).shape)

[[34]
 [19]
 [34]
 ...
 [32]
 [16]
 [37]] (23527, 1)
[[19]
 [39]
 [11]
 ...
 [20]
 [12]
 [22]] (5882, 1)
[[4]
 [4]
 [4]
 ...
 [8]
 [8]
 [8]] (7913, 1)
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39] (40,)
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39] (40,)
[0 1 2 3 4 5 6 7 8 9] (10,)


In [9]:
#params for trainval and test set
m_trainval = new_labels_trainval.shape[0]# number of instances in training set: 23527
print(m_trainval)

z_trainval = len(unique_labels_trainval)# number of classes in training set: 40
print(z_trainval)


n_test_seen = new_labels_test_seen.shape[0]# 5882
print(n_test_seen)

z1_test_seen = len(unique_labels_test_seen)# 40
print(z1_test_seen)


n_test_unseen = new_labels_test_unseen.shape[0]# 7913
print(n_test_unseen)

z1_test_unseen = len(unique_labels_test_unseen)# 10
print(z1_test_unseen)

23527
40
5882
40
7913
10


In [10]:
from tensorflow.keras.utils import to_categorical
gt_trainval = to_categorical(new_labels_trainval, z_trainval)

print(gt_trainval, gt_trainval.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]] (23527, 40)


In [11]:

#grountruth for trainval and test set
#gt_trainval = 0*np.ones((m_trainval, z_trainval))# 23527, 40
#gt_trainval[np.arange(m_trainval), np.squeeze(new_labels_trainval)] = 1

#print(gt_trainval, gt_trainval.shape)

In [12]:
input1_shape = trainval_vec.shape[1]
print(input1_shape)

attribute_shape = trainval_sig.shape[0]
print(attribute_shape)

output_shape = z_trainval
print(output_shape)


2048
85
40


In [13]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from keras.optimizers import SGD, Adam, Adagrad

# define model2 for attribute to class label mapping

input2 = Input(shape = attribute_shape)
output = Dense(output_shape, name="output", activation='softmax')(input2)

model2 = Model(inputs = input2, outputs = output)

#sgd = SGD(learning_rate = 1e-2, decay = 1e-6, momentum = 0.9, nesterov = True)
opt = Adam(learning_rate = 1e-2, beta_1=0.9, beta_2=0.999, epsilon=0.01, decay=0.0001)

model2.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

model2.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 85)]              0         
                                                                 
 output (Dense)              (None, 40)                3440      
                                                                 
Total params: 3,440
Trainable params: 3,440
Non-trainable params: 0
_________________________________________________________________


In [14]:
# define model1 for resnet feature to class label mapping

input1 = Input(shape = input1_shape)
#inter_pre = Dense(512, name="intermediate_previous", activation='relu')(input1)
inter = Dense(attribute_shape, name = "intermediate", activation = 'linear')(input1)
output = Dense(output_shape, name="output", activation='softmax')(inter)

model1 = Model(inputs = input1, outputs = output)

opt = Adam(learning_rate = 1e-2, beta_1=0.9, beta_2=0.999, epsilon=0.01, decay=0.0001)

model1.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])


model1.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 2048)]            0         
                                                                 
 intermediate (Dense)        (None, 85)                174165    
                                                                 
 output (Dense)              (None, 40)                3440      
                                                                 
Total params: 177,605
Trainable params: 177,605
Non-trainable params: 0
_________________________________________________________________


In [15]:
trainval_input1 = trainval_vec
print(trainval_input1.shape)

trainval_input2 = train_attributes
print(trainval_input2.shape)

trainval_output = gt_trainval
print(trainval_output.shape)

(23527, 2048)
(23527, 85)
(23527, 40)


In [16]:
from tensorflow.keras.utils import Sequence
class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y
    
batch_size = 16
from sklearn.model_selection import train_test_split    

In [17]:

X_train1, X_val1, y_train1, y_val1 = train_test_split(trainval_input1, trainval_output, test_size = 0.2, random_state = 42)

#train_gen1 = DataGenerator(X_train1, y_train1, batch_size)   
#val_gen1 = DataGenerator(X_val1, y_val1, batch_size)


X_train2, X_val2, y_train2, y_val2 = train_test_split(trainval_input2, trainval_output, test_size = 0.2, random_state = 42)

#train_gen2 = DataGenerator(X_train2, y_train2, batch_size)   
#val_gen2 = DataGenerator(X_val2, y_val2, batch_size)

In [18]:
iteration = 10
epochs1 = 100
epochs2 = 200

best_performance_micro = [0, 0, 0]
best_performance_macro = [0, 0, 0]

save_path = 'C:/Users/Admin/Sushree_Codes/Sush_3/Results/'
name = 'model1_AWA2_it10_100eph_adam_cce_16bch_1e-2lr_model2_adam_200'
        
    
for i in range(iteration):
    X_train2_it = X_train2[(len(X_train2)//iteration)*i:(len(X_train2)//iteration)*(i+1)]
    X_val2_it = X_val2[(len(X_val2)//iteration)*i:(len(X_val2)//iteration)*(i+1)]
    y_train2_it = y_train2[(len(y_train2)//iteration)*i:(len(y_train2)//iteration)*(i+1)]
    y_val2_it = y_val2[(len(y_val2)//iteration)*i:(len(y_val2)//iteration)*(i+1)]
    
    train_gen2 = DataGenerator(X_train2_it, y_train2_it, batch_size)   
    val_gen2 = DataGenerator(X_val2_it, y_val2_it, batch_size)

    train_summary2 = model2.fit(train_gen2, epochs = epochs2, verbose = 0, callbacks = None, validation_data = val_gen2, 
                              shuffle = True, steps_per_epoch = len(train_gen2)//batch_size, 
                              validation_steps = len(val_gen2)//batch_size)

    print("iteration:", i)
    print('model 2 is trained:', 'training acc:', train_summary2.history['accuracy'][-1], ',',  
          'training loss:', train_summary2.history['loss'][-1], ',', 
          'validation acc:', train_summary2.history['val_accuracy'][-1], ',',
         'validation_loss:', train_summary2.history['val_loss'][-1])

    weights_list2 = model2.get_weights()
    #print(weights_list2)

    model1.layers[-1].set_weights(weights_list2)

    X_train1_it = X_train1[(len(X_train1)//iteration)*i:(len(X_train1)//iteration)*(i+1)]
    X_val1_it = X_val1[(len(X_val1)//iteration)*i:(len(X_val1)//iteration)*(i+1)]
    y_train1_it = y_train1[(len(y_train1)//iteration)*i:(len(y_train1)//iteration)*(i+1)]
    y_val1_it = y_val1[(len(y_val1)//iteration)*i:(len(y_val1)//iteration)*(i+1)]
    
    train_gen1 = DataGenerator(X_train1_it, y_train1_it, batch_size)   
    val_gen1 = DataGenerator(X_val1_it, y_val1_it, batch_size)
    
    for layer in model1.layers[2:]:
        layer.trainable = True

    train_summary1 = model1.fit(train_gen1, epochs = epochs1, verbose = 0, callbacks = None, validation_data = val_gen1, 
                              shuffle = True, steps_per_epoch = len(train_gen1)//batch_size, 
                              validation_steps = len(val_gen1)//batch_size)
    
    #print("iteration:", i)
    print('model 1 is trained:', 'training acc:', train_summary1.history['accuracy'][-1], ',',  
          'training loss:', train_summary1.history['loss'][-1], ',', 
          'validation acc:', train_summary1.history['val_accuracy'][-1], ',',
         'validation_loss:', train_summary1.history['val_loss'][-1])
    
    weights_list1 = Model(inputs = model1.input, outputs = model1.layers[-1].output).get_weights()
    
    #predictions
    #outputs_seen = np.matmul(np.matmul(test_seen_vec, np.matmul(weights_list1[0], weights_list1[2])), test_seen_sig)
    outputs_seen = np.matmul(np.matmul(test_seen_vec, weights_list1[0]), test_seen_sig)
    
    preds_seen = np.array([np.argmax(output) for output in outputs_seen])
    
    cm_seen = confusion_matrix(new_labels_test_seen, preds_seen)
    #print(cm)
    # Compute macro average (averaging performance metrics by first calculating the metric separately for each class and 
    # then averaging these class-specific metrics)
    cm_seen_micro = cm_seen.astype('float') / cm_seen.sum(axis=1)[:, np.newaxis]
    #print(cm)
    avg_seen_micro = (sum(cm_seen_micro.diagonal())/len(unique_labels_test_seen))*100

    avg_seen_macro = (sum(cm_seen.diagonal())/len(new_labels_test_seen))*100
    
    #predictions
    outputs_unseen = np.matmul(np.matmul(test_unseen_vec, weights_list1[0]), test_unseen_sig)
    
    preds_unseen = np.array([np.argmax(output) for output in outputs_unseen])
    
    cm_unseen = confusion_matrix(new_labels_test_unseen, preds_unseen)
    # Compute macro average (averaging performance metrics by first calculating the metric separately for each class and 
    # then averaging these class-specific metrics)
    cm_unseen_micro = cm_unseen.astype('float') / cm_unseen.sum(axis=1)[:, np.newaxis]
    avg_unseen_micro = (sum(cm_unseen_micro.diagonal())/len(unique_labels_test_unseen))*100

    avg_unseen_macro = (sum(cm_unseen.diagonal())/len(new_labels_test_unseen))*100
    
    harmonic_micro = (2*avg_seen_micro*avg_unseen_micro) / (avg_seen_micro + avg_unseen_micro)
    harmonic_macro = (2*avg_seen_macro*avg_unseen_macro) / (avg_seen_macro + avg_unseen_macro)
    
    print('micro average')
    print('seen accuracy:', avg_seen_micro, 'unseen accuracy:', avg_unseen_micro, 'harmonic mean:', harmonic_micro)
    
    print('macro average')
    print('seen accuracy:', avg_seen_macro, 'unseen accuracy:', avg_unseen_macro, 'harmonic mean:', harmonic_macro)
    
    if harmonic_micro > best_performance_micro[2]:
        best_performance_micro = [avg_seen_micro, avg_unseen_micro, harmonic_micro]
        model1.save_weights(save_path + 'bw_micro_' + name + '.h5', overwrite=True)
        
    if harmonic_macro > best_performance_macro[2]:
        best_performance_macro = [avg_seen_macro, avg_unseen_macro, harmonic_macro]
        model1.save_weights(save_path + 'bw_macro_' + name + '.h5', overwrite=True)
        
    print('best accuracy micro','seen accuracy:', best_performance_micro[0], 'unseen accuracy:', best_performance_micro[1], 'harmonic mean:', best_performance_micro[2])
    print('best accuracy macro', 'seen accuracy:', best_performance_macro[0], 'unseen accuracy:', best_performance_macro[1], 'harmonic mean:', best_performance_macro[2])
    
    print('-----------------------------------------------------------------------------------------------------------')
    weights_list3 = model1.get_weights()
    model2.set_weights(weights_list3[2:])

iteration: 0
model 2 is trained: training acc: 0.9285714030265808 , training loss: 0.863211452960968 , validation acc: 0.8125 , validation_loss: 0.9085986614227295
model 1 is trained: training acc: 0.9642857313156128 , training loss: 0.8608346581459045 , validation acc: 0.8125 , validation_loss: 29.317358016967773
micro average
seen accuracy: 73.67236013527551 unseen accuracy: 49.18856976724767 harmonic mean: 58.99089367640173
macro average
seen accuracy: 76.91261475688542 unseen accuracy: 38.253506887400476 harmonic mean: 51.09449196210691
best accuracy micro seen accuracy: 73.67236013527551 unseen accuracy: 49.18856976724767 harmonic mean: 58.99089367640173
best accuracy macro seen accuracy: 76.91261475688542 unseen accuracy: 38.253506887400476 harmonic mean: 51.09449196210691
-----------------------------------------------------------------------------------------------------------
iteration: 1
model 2 is trained: training acc: 0.9910714030265808 , training loss: 0.3475036323070526 

In [19]:
gt_test_seen = to_categorical(new_labels_test_seen, z1_test_seen)

print(gt_test_seen, gt_test_seen.shape)

model1.evaluate(test_seen_vec, gt_test_seen)

p = model1.predict(test_seen_vec)

import tensorflow
cce = tensorflow.keras.losses.CategoricalCrossentropy()
cce(gt_test_seen, p).numpy()

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] (5882, 40)
184/184 [==============================] - 0s 778us/step


1.3452841

In [21]:
accuracy_seen_updated = 91.48
unseen_accuracy = 56.41
(2*accuracy_seen_updated*unseen_accuracy) / (accuracy_seen_updated + unseen_accuracy)

69.78682534316047